## Imports and Initialisations

In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import brown, stopwords

sents = brown.sents()
stopwords = set(stopwords.words('english'))
puncts = []

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sounak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def make_ordered_pairs(sentence):
    ans = list()
    for index, word in enumerate(sentence):
        temp = index
        while(temp != len(sentence) - 1):
            ans.append((sentence[index], sentence[temp]))
            temp += 1
    return ans

In [21]:
for sent in sents[:20]:
    filtered_sent = [w for w in sent if not w.lower() in stopwords]
    pairs = make_ordered_pairs(filtered_sent)
    print(pairs)

[('Fulton', 'Fulton'), ('Fulton', 'County'), ('Fulton', 'Grand'), ('Fulton', 'Jury'), ('Fulton', 'said'), ('Fulton', 'Friday'), ('Fulton', 'investigation'), ('Fulton', "Atlanta's"), ('Fulton', 'recent'), ('Fulton', 'primary'), ('Fulton', 'election'), ('Fulton', 'produced'), ('Fulton', '``'), ('Fulton', 'evidence'), ('Fulton', "''"), ('Fulton', 'irregularities'), ('Fulton', 'took'), ('Fulton', 'place'), ('County', 'County'), ('County', 'Grand'), ('County', 'Jury'), ('County', 'said'), ('County', 'Friday'), ('County', 'investigation'), ('County', "Atlanta's"), ('County', 'recent'), ('County', 'primary'), ('County', 'election'), ('County', 'produced'), ('County', '``'), ('County', 'evidence'), ('County', "''"), ('County', 'irregularities'), ('County', 'took'), ('County', 'place'), ('Grand', 'Grand'), ('Grand', 'Jury'), ('Grand', 'said'), ('Grand', 'Friday'), ('Grand', 'investigation'), ('Grand', "Atlanta's"), ('Grand', 'recent'), ('Grand', 'primary'), ('Grand', 'election'), ('Grand', 'pro